# Sprint12課題 深層学習スクラッチ畳み込みニューラルネットワーク1

NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

Sprint11で作成したディープニューラルネットワークのクラスを拡張する形でCNNを作成します。まず、Sprint12で1次元畳み込み層を作成し、畳み込みの基礎を理解することを目指します。そして、Sprint13で一般的に画像に対して使われる2次元畳み込み層とプーリング層を作成します。

### 1次元畳み込み層
畳み込みニューラルネットワークは画像に対して使われる2次元畳み込みが代表的ですが、理解を容易にするためにまずは1次元畳み込みを実装します。1次元畳み込みは系列データで使われることが多いです。畳み込みは任意の次元に対して考えることができ、立体データに対しての3次元畳み込みまでがフレームワークで一般的に用意されています。

### データセットの用意
引き続きMNISTデータセットを使用します。1次元畳み込みでは全結合のニューラルネットワークと同様に平滑化されたものを入力します。

### CNN分類器クラスの作成
1次元畳み込みニューラルネットワークモデルのクラスScratch1dCNNClassifierを作成してください。Sprint11で作成したScratchDeepNeuralNetrowkClassifierを元にしてください。

In [246]:
#ライブラリのインポート
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder


In [46]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [47]:
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

In [48]:
#float型へ
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)

#正規化
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

1.0
0.0


In [49]:
#データ分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
print(X_train.shape) # (48000, 784)
print(X_val.shape) # (12000, 784)

(48000, 784)
(12000, 784)


## 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成
チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成してください。基本構造はsprint11で作成したFCクラスと同じになります。なお、重みの初期化に関するクラスは必要に応じて作り変えてください。Xavierの初期値などを使う点は全結合層と同様です。

ここではパディングは考えず、ストライドも1に固定します。また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応してください。この部分の拡張はアドバンス課題とします。

フォワードプロパゲーションの数式は以下のようになります。

$$
a_i = \sum_{s=0}^{F-1}x_{(i+s)}w_s+b
$$

$a_i$ : 出力される配列のi番目の値

$F$ : フィルタのサイズ

$x_{(i+s)}$ : 入力の配列の(i+s)番目の値

$w_s$ : 重みの配列のs番目の値

$b$ : バイアス項

全てスカラーです。

次に更新式です。ここがAdaGradなどに置き換えられる点は全結合層と同様です。

$$
w_s^{\prime} = w_s - \alpha \frac{\partial L}{\partial w_s} \\
b^{\prime} = b - \alpha \frac{\partial L}{\partial b}
$$

$\alpha$ : 学習率

$\frac{\partial L}{\partial w_s}$ : $w_s$に関する損失$L$の勾配

$\frac{\partial L}{\partial b}$ : $b$に関する損失$L$の勾配

勾配$\frac{\partial L}{\partial w_s}$ や$\frac{\partial L}{\partial b}$ を求めるためのバックプロパゲーションの数式が以下です。

$$
\frac{\partial L}{\partial w_s} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}x_{(i+s)}\\
\frac{\partial L}{\partial b} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}
$$

$\frac{\partial L}{\partial a_i}$ :  勾配の配列のi番目の値

$N_{out}$ : 出力のサイズ

前の層に流す誤差の数式は以下です。

$$
\frac{\partial L}{\partial x_j} = \sum_{s=0}^{F-1} \frac{\partial L}{\partial a_{(j-s)}}w_s
$$

$\frac{\partial L}{\partial x_j}$: 前の層に流す誤差の配列のj番目の値

ただし、$j-s<0$  または$j-s>N_{out} - 1$のとき$\frac{\partial L}{\partial a_{(j-s)}} =0$です  
全結合層との大きな違いは、重みが複数の特徴量に対して共有されていることです。この場合は共有されている分の誤差を全て足すことで勾配を求めます。計算グラフ上での分岐はバックプロパゲーションの際に誤差の足し算をすれば良いことになります。

In [118]:
class SimpleConv1d:
    """
    Parameters
    ----------
    n_filters : int
        フィルター数
    filter_w : int
        フィルターの幅
    optimizer : instance
        最適化手法
    stride : int
        ストライド数
    pad : int
        パディング数
    
    Attribute
    ------------
    self.weight : ndarray, shape(n_filters, filter_w)
        重み（フィルター）
    self.bias : ndarray, shpae(n_channel)
    
    """
    def __init__(self, n_filters, filter_w, optimizer, stride, pad, w, b):
        
        self.n_filters = n_filters
        self.filter_w = filter_w
        self.optimizer = optimizer
        self.stride = stride
        self.pad = pad
        
        self.weight = w
        self.bias = b
        #self.weight = np.random.randn(n_filters, filter_w)
        #self.bias = np.zeros(1)
        

    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, width)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (n_samples, n_filters, out_w)
            出力
        """
        #バックワード用に保存
        self.forward_X = X.copy()
        
        #入力データのshape
        self.n, self.in_w = X.shape

        #1次元畳み込み後の出力サイズ
        self.out_w = output_size(self.in_w, self.filter_w, self.stride, self.pad)

        #出力格納用の空箱
        A = np.zeros([self.n, self.n_filters, self.out_w])
        
        #1d_convlution
        for sample in range(self.n):
            #フィルター枚数分繰り返し
            for fil in range(self.n_filters):
                #横のストライド
                for w_conv in range(self.out_w):
                    #出力用の配列の対応要素に加算していく。
                    A[sample][fil][w_conv] += np.sum(
                        X[sample][w_conv:w_conv+self.filter_w] * self.weight[fil]) + self.bias
                   
        return A
    
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (n_samples, n_filters, out_w)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (n_samples, width)
            前に流す勾配
        """
        #チャンネルを軸とした勾配合計
        self.dB = np.sum(dA)
        
        #deltaWの空箱
        self.dW = np.zeros(self.weight.shape)

        #deltaXの空箱
        dX = np.zeros([dA.shape[0], self.in_w])
        
        #サンプル数
        for sample in range(dA.shape[0]):
            #フィルター数
            for filters in range(self.n_filters):
                #横の出力サイズ
                for w_conv in range(self.out_w):
                    #横のフィルターサイズ
                    for w_filter in range(self.filter_w):

                        #deltaWの対応要素へ加算していく
                        self.dW[filters][w_filter] +=\
                            dA[sample][filters][w_conv] * self.forward_X[sample][w_conv+w_filter]

                        #deltaXの対応要素に加算していく
                        dX[sample][w_conv+w_filter] +=\
                            dA[sample][filters][w_conv] * self.weight[filters][w_filter]

                                    
        # 更新
        self = self.optimizer.update(self)
                      
        return dX
            



In [135]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    
    def __init__(self, lr):
        self.lr = lr
        
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス

        Returns
        ----------
        layer : 更新後の層のインスタンス
        """
        layer.weight -= self.lr * layer.dW
        layer.bias -= np.mean(self.lr * layer.dB)
        
        return layer

## 【問題2】1次元畳み込み後の出力サイズの計算
畳み込みを行うと特徴量の数が変化します。どのように変化するかは以下の数式から求められます。パディングやストライドも含めています。この計算を行う関数を作成してください。

$$
N_{out} =  \frac{N_{in}+2P-F}{S} + 1\\
$$

In [136]:
def output_size(in_n, filter_w, stride, pad):
    '''
    畳み込み後の出力サイズを計算する
    Parameters
    ----------
    in_n : int
        入力サイズ（特徴量の数）
    filter_w : int
        フィルタのサイズ
    stride : int
        ストライドのサイズ
    pad : パディングのサイズ
    
    Return
    ---------
    out_n : int
         出力のサイズ（特徴量の数）
    
    '''

    out_n = int(((in_n + 2 * pad - filter_w) / stride) + 1)

    return out_n

## 【問題3】小さな配列での1次元畳み込み層の実験
次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認してください。

入力x、重みw、バイアスbを次のようにします。

```python
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])
```

フォワードプロパゲーションをすると出力は次のようになります。

```python
a = np.array([35, 50])
```

次にバックプロパゲーションを考えます。誤差は次のようであったとします。
```python
delta_a = np.array([10, 20])
```

バックプロパゲーションをすると次のような値になります。
```python
delta_b = np.array([30])
delta_w = np.array([50, 80, 110])
delta_x = np.array([30, 110, 170, 140])
```

#### フォワードプロパゲーション

In [148]:
x = np.array([1,2,3,4])
w = np.array([3., 5., 7.])
b = np.array([1.])

In [149]:
optimizer = SGD(0.01)

In [150]:
sc1d = SimpleConv1d(
    n_filters=1, 
    filter_w=3, 
    optimizer=optimizer, 
    stride=1, 
    pad=0, 
    w=w.reshape(1,-1), 
    b=b
)

In [151]:
a = sc1d.forward(x.reshape(1,-1))
a

array([[[35., 50.]]])

#### バックプロパゲーション

In [152]:
delta_a = np.array([10, 20])

In [153]:
delta_x = sc1d.backward(delta_a.reshape(1,1,2))
delta_b = sc1d.dB
delta_w = sc1d.dW

In [154]:
print('delta_b : {}'.format(delta_b))
print('delta_w : {}'.format(delta_w))
print('delta_x : {}'.format(delta_x))

delta_b : 30
delta_w : [[ 50.  80. 110.]]
delta_x : [[ 30. 110. 170. 140.]]


## 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成
チャンネル数を1に限定しない1次元畳み込み層のクラスConv1dを作成してください。

紙やホワイトボードを使い計算グラフを書きながら考えてください。

例えば以下のようなx, w, bがあった場合は、

```python
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）
```

出力は次のようになります。
```python
a = np.array([[16, 22], [17, 23], [18, 24]]) # shape(3, 2)で、（出力チャンネル数、特徴量数）である。
```

入力が2チャンネル、出力が3チャンネルの例です。計算グラフを書いた上で、バックプロパゲーションも手計算で考えてみましょう。計算グラフの中には和と積しか登場しないので、微分を新たに考える必要はありません。

**補足**

チャンネル数を加える場合、配列をどういう順番にするかという問題があります。(バッチサイズ、チャンネル数、特徴量数)または(バッチサイズ、特徴量数、チャンネル数)が一般的で、ライブラリによって順番は異なっています。（切り替えて使用できるものもあります）

今回のスクラッチでは自身の実装上どちらが効率的かを考えて選んでください。上記の例ではバッチサイズは考えておらず、(チャンネル数、特徴量数)です。

In [248]:
class Conv1d:
    """
    Parameters
    ----------
    n_filters : int
        フィルター数
    n_channel : int
        チャンネル数
    filter_w : int
        フィルターの幅
    optimizer : instance
        最適化手法
    stride : int
        ストライド数
    pad : int
        パディング数
    
    Attribute
    ------------
    self.weight : ndarray, shape(n_filters, n_channel, filter_w)
        重み（フィルター）
    self.bias : ndarray, shpae(n_channel)
    
    """
    def __init__(self, n_filters, n_channel, filter_w, optimizer, stride, pad, w=None, b=None):
        
        self.n_filters = n_filters
        self.n_channel = n_channel
        self.filter_w = filter_w
        self.optimizer = optimizer
        self.stride = stride
        self.pad = pad
        
        if w is None:
            self.weight = np.random.randn(n_filters, n_channel, filter_w)
        else:
            self.weight = w
            
        if b is None:
            self.bias = np.zeros(n_channel)
        else:
            self.bias = b
        

    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_channels, width)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (n_samples, n_filters, out_w)
            出力
        """
        #バックワード用に保存
        self.forward_X = X.copy()
        
        #入力データのshape
        self.n, self.in_c, self.in_w = X.shape

        #1次元畳み込み後の出力サイズ
        self.out_w = output_size(self.in_w, self.filter_w, self.stride, self.pad)

        #出力格納用の空箱
        A = np.zeros([self.n, self.n_filters, self.out_w])
        
        #1d_convlution
        for sample in range(self.n):
            #フィルター枚数分繰り返し
            for fil in range(self.n_filters):
                #チャンネル数分繰り返し
                for channel in range(self.in_c):
                    #横のストライド
                    for w_conv in range(self.out_w):
                        #出力用の配列の対応要素に加算していく。
                        A[sample][fil][w_conv] += np.sum(
                            X[sample][channel][w_conv:w_conv+self.filter_w] 
                            * self.weight[fil][channel]) 
            A += self.bias.reshape(1, -1, 1)
                   
        return A
    
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (n_samples, n_filters, out_w)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (n_samples, n_channels, width)
            前に流す勾配
        """
        #チャンネルを軸とした勾配合計
        self.dB = np.sum(dA, axis=(0,2))
        
        #deltaWの空箱
        self.dW = np.zeros(self.weight.shape)

        #deltaXの空箱
        dX = np.zeros([dA.shape[0], self.n_channel, self.in_w])
        
        #サンプル数
        for sample in range(dA.shape[0]):
            #チャンネル数
            for channel in range(self.n_channel):
                #フィルター数
                for filters in range(self.n_filters):
                    #横の出力サイズ
                    for w_conv in range(self.out_w):
                        #横のフィルターサイズ
                        for w_filter in range(self.filter_w):

                            #deltaWの対応要素へ加算していく
                            self.dW[filters][channel][w_filter] +=\
                                dA[sample][filters][w_conv] *\
                                self.forward_X[sample][channel][w_conv+w_filter]

                            #deltaXの対応要素に加算していく
                            dX[sample][channel][w_conv+w_filter] +=\
                                dA[sample][filters][w_conv] *\
                                self.weight[filters][channel][w_filter]

                                    
        # 更新
        self = self.optimizer.update(self)
                      
        return dX
            



In [249]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1., 2., 3.]) # （出力チャンネル数）

In [250]:
conv1d = Conv1d(
    n_filters=3, 
    n_channel=2, 
    filter_w=3, 
    optimizer=optimizer, 
    stride=1, 
    pad=0, 
    w=w, 
    b=b
)

In [251]:
a = conv1d.forward(x.reshape(1,2,4))
a

array([[[16., 22.],
        [17., 23.],
        [18., 24.]]])

In [252]:
conv1d.backward(a)

array([[[ 51., 120., 120.,  69.],
        [ 51., 120., 120.,  69.]]])

## 【問題5】学習・推定
これまで使ってきたニューラルネットワークの全結合層の一部をConv1dに置き換えて学習と推定を行ってください。出力層だけは全結合層をそのまま使ってください。

チャンネルが複数ある状態では全結合層への入力は行えません。その段階でのチャンネルは1になるようにするか、平滑化を行います。平滑化はNumPyのreshapeが使用できます。

[numpy.reshape — NumPy v1.15 Manual](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.reshape.html)

画像に対しての1次元畳み込みは実用上は行わないことのため、精度は問いません。

In [275]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    
    Attribute
    ------------
    self.W : 重み
    self.B : バイアス
    self.H_w : 前のイテレーションまでの勾配の(重み)二乗和(初期値0)
    self.H_b : 前のイテレーションまでの勾配の(バイアス)二乗和(初期値0)
    self.forward_Z : forward時の入力値(backward用に利用)
    self.dW : 重みの勾配
    self.dB : バイアスの勾配
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.weight = initializer.W(n_nodes1, n_nodes2)
        self.bias = initializer.B(n_nodes2)
                
        self.forward_Z=None

    def forward(self, Z):
        """
        フォワード
        Parameters
        ----------
        Z : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        #backfoward用に保存
        self.forward_Z = Z.copy()
        
        A = (Z @ self.weight) + self.bias
        
        return A
    
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        self.dB = dA.copy()
        self.dW = self.forward_Z.T @ dA
        
        #[batch_size, n_nodes2] dot [n_nodes2, n_nodes1]
        #→ [batch_size, n_nodes1]
        dZ = dA @ self.weight.T 
        
        # 更新
        self = self.optimizer.update(self)
              
        return dZ


In [276]:
class MaxPool1D:
    '''
    最大プーリング層
    Parameters
    ----------
    stride_w : int
        横のストライドサイズ
    
    Attribute
    -------------
    self.forwad_X : ndarray, shape (n_samples, n_channels, width)
        フォワード時の入力値データ（バックワード用）
    self.out_w : int
        プーリング時の出力時の幅
    self.pool_out_idx :  ndarray, shape (n_samples, n_filters, out_w)
        プーリング時の最大値インデックス
    '''
    
    def __init__(self, stride_w):
        
        self.stride_w = stride_w
    
        
    def forward(self, X):
        '''
        プーリングを行い、ダウンサンプリングする
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_channels, width)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (n_samples, n_filters, out_w)
            出力
        '''  
        #バックワード用にコピー
        self.forward_X = X.copy()
        
        #入力データのshape
        N, C , in_w = X.shape

        #出力サイズの計算
        self.out_w = int(in_w/self.stride_w)

        #出力データ(プーリング結果)の空箱
        A = np.zeros([N, C, self.out_w])
        #プーリング時の最大値インデックス記憶する空箱
        self.pool_out_idx = np.zeros([N, C, self.out_w])
        
        for sample in range(N):
            for channel in range(C):
                for w_pool in range(self.out_w):

                    #プーリング結果を出力配列に更新していく
                    A[sample][channel][w_pool] = \
                        np.max(X[sample][channel]
                               [w_pool*self.stride_w:(w_pool+1)*self.stride_w])

                    #プーリング時の最大値インデックスを記憶していく
                    self.pool_out_idx[sample][channel][w_pool] = \
                        np.argmax(X[sample][channel]
                                  [w_pool*self.stride_w:(w_pool+1)*self.stride_w])
                        
        return A
    
    def backward(self, dA):
        '''
        プーリングのバックワード、アップサンプリングする
        Parameters
        ----------
        dA : 次の形のndarray, shape (n_samples, n_filters, out_w)
            後ろから流れてきた勾配
        Returns
        ----------
        dX : 次の形のndarray, shape (n_samples, n_channels, width)
            前に流す勾配
        '''
        #入力データのサンプル数、チャンネル数を確認
        N = dA.shape[0]
        C = dA.shape[1]
        
        #出力データの空箱を作成する
        dX = np.zeros(self.forward_X.shape)

        for sample in range(N):
            for channel in range(C):
                for w_pool in range(self.out_w):

                    #最大値インデックスより、ストライド配列内の列を特定する
                    column = int(self.pool_out_idx[sample][channel][w_pool])

                    #フォワード前と同じインデックスのみに対して更新を行う
                    dX[sample][channel][w_pool*self.stride_w+column]\
                        = dA[sample][channel][w_pool]
    
        
        
        return dX
        

In [277]:
class Relu():
    """
    ReLUの計算
    
    Parameters
    -----------
    
    Attribute
    -----------
    self.mask : 入力値の0以下を判定するboolリスト
    
    """
    
    def __init__(self):
        self.mask = None     
        
    def forward(self, A):
        """
        フォワードにおける活性化関数の計算
        
        Parameters
        -----------
        A : 活性化関数計算前
        
        Return
        -----------
        Z : 出力

        """
        self.mask = (A <= 0)
        
        Z = A.copy()
        
        Z[self.mask] = 0
        
        return Z
    
    def backward(self, dZ):
        """
        バックワードにおける活性化関数の計算
        
        Parameters
        -----------
        dZ : 活性化関数計算前
        
        Return
        -----------
        dA : 出力

        """        
        dA = dZ.copy()
        
        dA[self.mask] = 0
        
            
        return dA

In [278]:
class Softmax():
    """
    softmaxの計算
    
    Parameters
    -----------
    
    Attribute
    -----------
    self.cost : 交差エントロピー誤差を格納
    """
    
    def __init__(self):
        self.cost = None
        
    def forward(self, A):
        """
        フォワードにおけるソフトマックスの計算
        
        Parameters
        -----------
        A : 活性化関数計算前
        
        Return
        -----------
        Z : 出力
        """
        #オーバーフロー対策
        max_A = np.max(A)

        #最大要素を引いてからexpをかけることでオーバーフローを回避
        exp_A = np.exp(A - max_A)

         #分母を計算
        sum_exp_A = np.sum(exp_A, axis=1).reshape(-1, 1)
        
        Z = exp_A / sum_exp_A

        return Z
        
    
    def backward(self, Z, Y):
        """
        バックワードにおけるソフトマックスと交差エントロピー誤差
        
        Parameters
        -----------
        Z : 出力層で計算された出力
        Y : 正解値
        
        Return
        -----------
        dA : 出力

        """
        #交差エントロピー誤差
        self.cost = - np.sum(Y * np.log(Z), axis=1)

        #バックワード(出力層)
        dA = Z - Y
        
        return dA
    


In [279]:
class HeInitializer:
    """
    Heによる初期化
    
    Parameters
    ----------
    
    """
    
    def __init__(self):
        pass
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = np.random.randn(n_nodes1, n_nodes2) / np.sqrt(n_nodes1) * np.sqrt(2)

        return W
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B = np.zeros(n_nodes2)

        return B

In [280]:
class Flatten:
    '''
    平滑化を行う
    
    Parameters
    ----------
    None
    
    Attribute
    -------------
    self.f_c : int
        フォワード時の入力データチャンネル数
    self.f_w : int
        フォワード時の入力データ幅
    '''

    
    def __init__(self):
        self.f_c = None
        self.f_w = None
    
    def forward(self, X):
        '''
        入力データに対して平滑化を行い、3次元→2次元にする
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_channels, width)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (n_samples, n_channels * width)
            出力
        '''  
        #入力データの形
        f_n, self.f_c, self.f_w = X.shape
        
        fX = X.reshape(f_n, self.f_c * self.f_w)
        
        return fX
        
        
    def backward(self, fX):
        '''
        フォワードにて平滑化された入力値のshapeを元に戻す
        Parameters
        ----------
        fxX : 次の形のndarray, shape (n_samples, n_channels * width)
            平滑化された入力値
        Returns
        ----------
        A : 次の形のndarray, shape (n_samples, n_channels, width)
            出力（元に戻す）
        '''  
        #サンプル数を確認
        b_n = fX.shape[0]
        
        #元の形に戻す
        rX = fX.reshape(b_n, self.f_c, self.f_w)
        
        return rX
    
    

In [300]:
class Scratch1dCNNClassifier():
    """
    1次元畳み込みニューラルネットワーク
    (sgd)
    conv2d - relu - pooling - relu - FC - softmax

    Parameters
    ----------
    verbose : bool
        学習過程を出力する場合はTrue
    n_epochs : int(default:30)
        イテレーション数
    lr : flaot (default:1e-3)
        学習率
    batch : int(default :10)
        ミニバッチの単位数
    conv_n_filters : int(default:3)
        畳み込み時のフィルター枚数
    conv_n_channel : int(default:1)
        畳み込み時のチャンネル数
    conv_filter_w : int(default:3)
        畳み込み時のフィルターサイズ(横)
    conv_stride : int(default:1)
        畳み込み時のストライド数
    conv_pad : int(default:0)
        畳み込み時のパディング数
    pool_stride_w : int(default:2)
        プーリング時のストライドサイズ(横)

    Attributes
    ----------
    self.loss : ndarray,shape(n_epochs, )
        エポックごとの誤差を格納
    self.val_loss : ndarray,shape(n_epochs, )
        エポックごとの誤差(検証用データ)を格納
        
        
    """

    def __init__(self, verbose = True, n_epochs=10, lr=1e-3, batch=10, 
                 conv_n_filters=3, conv_n_channel=1, conv_filter_w=3, 
                 conv_stride=1, conv_pad=0, pool_stride_w=2):

        self.verbose = verbose                            #True(default):学習過程を表示、False:非表示
        self.n_epochs = n_epochs                       #エポック数(default:30)
        self.lr = lr                                               #学習率(default:1e-3)
        self.batch = batch                                  #ミニバッチを行うサイズ(default:10)
        self.conv_n_filters = conv_n_filters          #畳み込み時のフィルター枚数
        self.conv_n_channel = conv_n_channel   #畳み込み時のチャンネル数
        self.conv_filter_w = conv_filter_w           #畳み込み時のフィルターサイズ(横)
        self.conv_stride = conv_stride               #畳み込み時ストライド数
        self.conv_pad = conv_pad                     #畳み込み時のパディング数
        self.pool_stride_w = pool_stride_w         #プーリング時のストライド数(横)

        self.loss = np.zeros(n_epochs)
        self.val_loss = np.zeros(n_epochs)
    
    
    def fit(self, X, y, X_val=None, y_val=None):
        """
        ニューラルネットワーク分類器を学習する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            学習用データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            学習用データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証用データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証用データの正解値
        """
        #出力層のアウトプットノード数
        self.n_output = np.unique(y).shape[0]
        
        #conv後の出力サイズ
        conv_out_w = int(((X.shape[2] + 2 * self.conv_pad - self.conv_filter_w) // self.conv_stride) + 1)
        
        #pooling後の出力サイズ
        pool_out_w = int(conv_out_w // self.pool_stride_w)
        
        #出力層への入力サイズ
        n_node = self.conv_n_filters * pool_out_w 
        
        #正解データをワンホットエンコーディング
        y = self._onehot(y)
        #検証用データもあればワンホット
        if X_val is not None:
            y_val = self._onehot(y_val)


        #入力層のノード数
        self.n_features = X.shape[1]

        #minibatchデータを生成
        train_minibatch = GetMiniBatch(X, y, batch_size=self.batch)
        

        #最適化手法のインスタンスを選択、生成。
        optimizer = SGD(self.lr)
        
        #活性化関数
        
        #1層目(conv1d)
        #インスタンス生成、重みの初期化
        self.conv1d = Conv1d(self.conv_n_filters, self.conv_n_channel, 
                         self.conv_filter_w, optimizer, self.conv_stride, self.conv_pad)
        self.activation1 = Relu()

        #pooling
        #インスタンス生成
        self.pool1d= MaxPool1D(self.pool_stride_w)
        self.activation2 = Relu()

        #3層目(全結合　出力層)
        #インスタンス生成、重みの初期化
        self.FC = FC(n_node, self.n_output, HeInitializer(), optimizer)
        self.activation3 = Softmax()
        
        #平滑化インスタンス生成
        self.fl = Flatten()

        #エポック数だけトレーニングを繰り返す
        for epoch in range(self.n_epochs):
                            
            #引数で設定したminibatch数の単位で学習を行う
            for mini_X, mini_y in train_minibatch:
                X = mini_X.copy()
                Y = mini_y.copy()
                                            
                #フォワードプロパゲーション
                A1 = self.conv1d.forward(X)
                Z1 = self.activation1.forward(A1)
                A2 = self.pool1d.forward(Z1)
                Z2 = self.activation2.forward(A2)
                Z2 = self.fl.forward(Z2)
                A3 = self.FC.forward(Z2)
                Z3 = self.activation3.forward(A3)
                                
                #バックプロパゲーション
                dA3 = self.activation3.backward(Z3, Y) # 交差エントロピー誤差とソフトマックスを合わせている
                dZ2 = self.FC.backward(dA3)
                dZ2 = self.fl.backward(dZ2)
                dA2 = self.activation2.backward(dZ2)
                dZ1 = self.pool1d.backward(dA2)
                dA1 = self.activation1.backward(dZ1)
                dZ0 = self.conv1d.backward(dA1) # dZ0は使用しない
                    
            ############
            # 評価
            ############
            #誤差を格納
            self.loss[epoch] = np.mean(self.activation3.cost)
                        
            #検証用データが引数にある場合、処理を行う
            if X_val is not None:
                                
                #フォワードプロパゲーション
                A1_val = self.conv1d.forward(X_val)
                Z1_val = self.activation1.forward(A1_val)
                A2_val = self.pool1d.forward(Z1_val)
                Z2_val = self.activation2.forward(A2_val)
                Z2_val = self.fl.forward(Z2_val)
                A3_val = self.FC.forward(Z2_val)
                Z3_val = self.activation3.forward(A3_val)
                
                #検証用データの交差エントロピー誤差を計算
                cost_val = self._compute_cost(y_val, Z3_val)

                #誤差を格納
                self.val_loss[epoch] = np.mean(cost_val)
                            

            #verboseをTrueにした際は学習過程などを出力する
            if self.verbose:
                #一度だけ、'Cross Entropy Error'を出力
                if epoch == 0:
                    print('Cross Entropy Error')
                    
                #エポックごとのコスト関数を出力
                print('epoch{} : {}'.format(epoch+1, np.mean(self.activation3.cost)))
                
                #検証用データがある場合、そのコスト関数も出力
                if X_val is not None:
                    print('epoch_val{} : {}'.format(epoch+1, np.mean(cost_val)))

        return self


    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
        y_pred :  次の形のndarray, shape (n_samples, 1)
            推定結果
        """        
        #フォワードプロパゲーション
        A1 = self.conv1d.forward(X)
        Z1 = self.activation1.forward(A1)
        A2 = self.pool1d.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.FC.forward(Z2)
        Z3 = self.activation3.forward(A3)

        
        #出力層の確率から、最大値をそのクラスとする
        y_pred = np.argmax(Z3, axis=1)
        
        return y_pred
    
    def _onehot(self, y):
        """
        多クラス分類を行う際のone-hot表現に変換

        Parameters
        ----------
        y : 次の形のndarray, shape (n_samples, )
            サンプル

        Returns
        -------
        y_one_hot : 次の形のndarray, shape (n_samples, n_classes)
            推定結果
        """
        enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
        y_one_hot = enc.fit_transform(y[:, np.newaxis])
        
        return y_one_hot
    
    
    #交差エントロピー誤差
    def _compute_cost(self, y, y_pred):

        return - np.sum(y * np.log(y_pred), axis=1)
    
    
    
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop

    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [301]:
s1dcnn = Scratch1dCNNClassifier(
    verbose = True, 
    n_epochs=10, 
    lr=1, 
    batch=10, 
    conv_n_filters=1, 
    conv_n_channel=1, 
    conv_filter_w=3, 
    conv_stride=1, 
    conv_pad=0, 
    pool_stride_w=2
)


In [302]:
#処理に時間がかかるため、入力データサンプル数を制限
s1dcnn.fit(X_train.reshape(-1,1,784)[:300], y_train[:300],X_val.reshape(-1,1,784)[0:100], y_val[0:100])

Cross Entropy Error
epoch1 : 2.302585092994046
epoch_val1 : 2.3025850929940455
epoch2 : 2.302585092994046
epoch_val2 : 2.3025850929940455
epoch3 : 2.302585092994046
epoch_val3 : 2.3025850929940455
epoch4 : 2.302585092994046
epoch_val4 : 2.3025850929940455
epoch5 : 2.302585092994046
epoch_val5 : 2.3025850929940455
epoch6 : 2.302585092994046
epoch_val6 : 2.3025850929940455
epoch7 : 2.302585092994046
epoch_val7 : 2.3025850929940455
epoch8 : 2.302585092994046
epoch_val8 : 2.3025850929940455
epoch9 : 2.302585092994046
epoch_val9 : 2.3025850929940455
epoch10 : 2.302585092994046
epoch_val10 : 2.3025850929940455


※精度が上がらず。。

## 【問題6】（アドバンス課題）パディングの実装
畳み込み層にパディングを加えてください。1次元配列の場合、前後にn個特徴量を増やせるようにしてください。

最も単純なパディングは全て0で埋めるゼロパディングであり、CNNでは一般的です。他に端の値を繰り返す方法などもあります。

フレームワークによっては、元の入力のサイズを保つようにという指定をすることができます。この機能も持たせておくと便利です。

なお、NumPyにはパディングの関数が存在します。

numpy.pad — NumPy v1.15 Manual

**※省略**

## 【問題7】（アドバンス課題）ミニバッチへの対応
ここまでの課題はバッチサイズ1で良いとしてきました。しかし、実際は全結合層同様にミニバッチ学習が行われます。Conv1dクラスを複数のデータが同時に計算できるように変更してください。

**問題5にて実装済み**

## 【問題8】（アドバンス課題）任意のストライド数
ストライドは1限定の実装をしてきましたが、任意のストライド数に対応できるようにしてください。

**問題5にて実装済み**